# BSM2 Flowsheet tutorial: 

### This demonstration will show 
- Part 1: how to build, initialize, and simulate the flowsheet for Benchmark Simulation Model 2 (BSM2). 
- Part 2: Demonstrate optimization and sensitivity analysis over specific parameters of the flowsheet (pending).
- Useful Links:
    - Public Github Repository: https://github.com/watertap-org/watertap
    - Documentation: https://watertap.readthedocs.io/en/stable/
    - Activated Sludge Model No. 1 (ASM1) documentation: https://watertap.readthedocs.io/en/stable/technical_reference/property_models/ASM1.html
    - Anaerobic Digestion Model No. 1 (ADM1) documentation: https://watertap.readthedocs.io/en/stable/technical_reference/property_models/ADM1.html
    - ASM1-ADM1 Translator documentation: https://watertap.readthedocs.io/en/stable/technical_reference/unit_models/translators/translator_asm1_adm1.html
    - ADM1-ASM1 Translator documentation: https://watertap.readthedocs.io/en/stable/technical_reference/unit_models/translators/translator_adm1_asm1.html    
    - Unit Model documentation: https://watertap.readthedocs.io/en/stable/technical_reference/unit_models/index.html
    - BSM2 flowsheet code: https://github.com/watertap-org/watertap/blob/main/watertap/examples/flowsheets/case_studies/full_water_resource_recovery_facility/BSM2.py


# Part 1: Build, setup, and simulate the Benchmark Simulation Model

<img src="BSM2.png" width="1000" height="680">

## Step 1: Import libraries from Pyomo, IDAES, and WaterTAP.

### Step 1.1: Import some essentials from Pyomo and IDAES:

In [1]:
import pyomo.environ as pyo
from pyomo.network import Arc, SequentialDecomposition
from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
from watertap.core.solvers import get_solver
import idaes.core.util.scaling as iscale

### Step 1.2: Import unit models from WaterTAP and IDAES:

In [2]:
# Import anaerobic digester model
from watertap.unit_models.anaerobic_digester import AD

# Import aeration tank model
from watertap.unit_models.aeration_tank import AerationTank, ElectricityConsumption

# Import continuous stirred tank reactor model
from watertap.unit_models.cstr import CSTR

# Import clarifier model
from watertap.unit_models.clarifier import Clarifier

# Import BSM2 separator models 
from watertap.unit_models.thickener import Thickener
from watertap.unit_models.dewatering import DewateringUnit

# Import idaes unit models for separators and mixers and ASM models
from idaes.models.unit_models.mixer import MomentumMixingType
from idaes.models.unit_models import (
    Feed,
    Mixer,
    Separator,
    PressureChanger,
    Product,
)
from idaes.models.unit_models.separator import SplittingType

# import translator models from WaterTAP
from watertap.unit_models.translators.translator_asm1_adm1 import Translator_ASM1_ADM1
from watertap.unit_models.translators.translator_adm1_asm1 import Translator_ADM1_ASM1

### Step 1.3: Import all BSM2 required property models

Property blocks are an important building block in WaterTap as they are a Python class which contain information on units, physical properties, etc.

In [3]:
# Import Anaerobic Digester Model properties 
from watertap.property_models.unit_specific.anaerobic_digestion.adm1_properties import (
    ADM1ParameterBlock,
)
from watertap.property_models.unit_specific.anaerobic_digestion.adm1_reactions import (
    ADM1ReactionParameterBlock,
)
from watertap.property_models.unit_specific.anaerobic_digestion.adm1_properties_vapor import (
    ADM1_vaporParameterBlock,
)

# Import Activated Sludge Model properties 
from watertap.property_models.unit_specific.activated_sludge.asm1_properties import (
    ASM1ParameterBlock,
)
from watertap.property_models.unit_specific.activated_sludge.asm1_reactions import (
    ASM1ReactionParameterBlock,
)

## Step 2 Flowsheet building

## Step 2.1: Create Flowsheet
We will start by creating a pyomo model and a flowsheet

In [4]:
m = pyo.ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False)

We then include all the necessary property blocks we imported into the flowsheet. Namely, we include the ASM1 and ADM1 models, which are separated into their respective property and reaction models. Additionally, the vapor phase of ADM1 was separated into its own property model.

In [5]:
m.fs.props_ASM1 = ASM1ParameterBlock()
m.fs.props_ADM1 = ADM1ParameterBlock()
m.fs.props_vap = ADM1_vaporParameterBlock()
m.fs.ADM1_rxn_props = ADM1ReactionParameterBlock(property_package=m.fs.props_ADM1)
m.fs.ASM1_rxn_props = ASM1ReactionParameterBlock(property_package=m.fs.props_ASM1)

### Step 2.2: Setup Activated Sludge process

We will start by setting up the activated sludge process unit models and connectivity.

First, we set up a Feed model for our feed stream and will name it `Feedwater`. 

In [6]:
# Feed water stream
m.fs.FeedWater = Feed(property_package=m.fs.props_ASM1)
# Mixer for feed water and recycled sludge
m.fs.MX1 = Mixer(
    property_package=m.fs.props_ASM1, inlet_list=["feed_water", "recycle"], momentum_mixing_type=MomentumMixingType.equality,
)
# First reactor (anoxic) - standard CSTR
m.fs.R1 = CSTR(
    property_package=m.fs.props_ASM1, reaction_package=m.fs.ASM1_rxn_props
)
# Second reactor (anoxic) - standard CSTR
m.fs.R2 = CSTR(
    property_package=m.fs.props_ASM1, reaction_package=m.fs.ASM1_rxn_props
)
# Third reactor (aerobic) - CSTR with injection
m.fs.R3 = AerationTank(
        property_package=m.fs.props_ASM1,
        reaction_package=m.fs.ASM1_rxn_props,
        electricity_consumption=ElectricityConsumption.calculated,
)
# Fourth reactor (aerobic) - CSTR with injection
m.fs.R4 = AerationTank(
        property_package=m.fs.props_ASM1,
        reaction_package=m.fs.ASM1_rxn_props,
        electricity_consumption=ElectricityConsumption.calculated,
)
# Fifth reactor (aerobic) - CSTR with injection
m.fs.R5 = AerationTank(
        property_package=m.fs.props_ASM1,
        reaction_package=m.fs.ASM1_rxn_props,
        electricity_consumption=ElectricityConsumption.calculated,
)
m.fs.SP5 = Separator(
    property_package=m.fs.props_ASM1, outlet_list=["underflow", "overflow"]
)
# Clarifier
m.fs.CL1 = Clarifier(
    property_package=m.fs.props_ASM1,
    outlet_list=["underflow", "effluent"],
    split_basis=SplittingType.componentFlow,
)
# Sludge purge splitter
m.fs.SP6 = Separator(
    property_package=m.fs.props_ASM1,
    outlet_list=["recycle", "waste"],
    split_basis=SplittingType.totalFlow,
)
# Mixing sludge recycle and R5 underflow
m.fs.MX6 = Mixer(
    property_package=m.fs.props_ASM1, inlet_list=["clarifier", "reactor"], momentum_mixing_type=MomentumMixingType.equality
)
# Product Blocks
m.fs.Treated = Product(property_package=m.fs.props_ASM1)
# Recycle pressure changer - use a simple isothermal unit for now
m.fs.P1 = PressureChanger(property_package=m.fs.props_ASM1)

Secondly we will use pyomo arcs as streams connecting unit to unit

In [7]:
# Link units
m.fs.stream2 = Arc(source=m.fs.MX1.outlet, destination=m.fs.R1.inlet)
m.fs.stream3 = Arc(source=m.fs.R1.outlet, destination=m.fs.R2.inlet)
m.fs.stream4 = Arc(source=m.fs.R2.outlet, destination=m.fs.R3.inlet)
m.fs.stream5 = Arc(source=m.fs.R3.outlet, destination=m.fs.R4.inlet)
m.fs.stream6 = Arc(source=m.fs.R4.outlet, destination=m.fs.R5.inlet)
m.fs.stream7 = Arc(source=m.fs.R5.outlet, destination=m.fs.SP5.inlet)
m.fs.stream8 = Arc(source=m.fs.SP5.overflow, destination=m.fs.CL1.inlet)
m.fs.stream9 = Arc(source=m.fs.SP5.underflow, destination=m.fs.MX6.reactor)
m.fs.stream10 = Arc(source=m.fs.CL1.effluent, destination=m.fs.Treated.inlet)
m.fs.stream11 = Arc(source=m.fs.CL1.underflow, destination=m.fs.SP6.inlet)
m.fs.stream13 = Arc(source=m.fs.SP6.recycle, destination=m.fs.MX6.clarifier)
m.fs.stream14 = Arc(source=m.fs.MX6.outlet, destination=m.fs.P1.inlet)
m.fs.stream15 = Arc(source=m.fs.P1.outlet, destination=m.fs.MX1.recycle)
pyo.TransformationFactory("network.expand_arcs").apply_to(m)

Next we will set the conditions for the inlet water

In [8]:
# Feed Water Conditions
m.fs.FeedWater.flow_vol.fix(20648 * pyo.units.m**3 / pyo.units.day)
m.fs.FeedWater.temperature.fix(308.15 * pyo.units.K)
m.fs.FeedWater.pressure.fix(1 * pyo.units.atm)
m.fs.FeedWater.conc_mass_comp[0, "S_I"].fix(
    27 * pyo.units.g / pyo.units.m**3
)
m.fs.FeedWater.conc_mass_comp[0, "S_S"].fix(
    58 * pyo.units.g / pyo.units.m**3
)
m.fs.FeedWater.conc_mass_comp[0, "X_I"].fix(92 * pyo.units.g / pyo.units.m**3)
m.fs.FeedWater.conc_mass_comp[0, "X_S"].fix(
    363 * pyo.units.g / pyo.units.m**3
)
m.fs.FeedWater.conc_mass_comp[0, "X_BH"].fix(
    50 * pyo.units.g / pyo.units.m**3
)
m.fs.FeedWater.conc_mass_comp[0, "X_BA"].fix(0 * pyo.units.g / pyo.units.m**3)
m.fs.FeedWater.conc_mass_comp[0, "X_P"].fix(0 * pyo.units.g / pyo.units.m**3)
m.fs.FeedWater.conc_mass_comp[0, "S_O"].fix(0 * pyo.units.g / pyo.units.m**3)
m.fs.FeedWater.conc_mass_comp[0, "S_NO"].fix(0 * pyo.units.g / pyo.units.m**3)
m.fs.FeedWater.conc_mass_comp[0, "S_NH"].fix(
    23 * pyo.units.g / pyo.units.m**3
)
m.fs.FeedWater.conc_mass_comp[0, "S_ND"].fix(
    5 * pyo.units.g / pyo.units.m**3
)
m.fs.FeedWater.conc_mass_comp[0, "X_ND"].fix(
    16 * pyo.units.g / pyo.units.m**3
)
m.fs.FeedWater.alkalinity.fix(7 * pyo.units.mol / pyo.units.m**3)

Finally for the activated sludge models we will set the conditions for the unit models. We will start with the reactors volume

In [9]:
# Reactor sizing
m.fs.R1.volume.fix(1000 * pyo.units.m**3)
m.fs.R2.volume.fix(1000 * pyo.units.m**3)
m.fs.R3.volume.fix(1333 * pyo.units.m**3)
m.fs.R4.volume.fix(1333 * pyo.units.m**3)
m.fs.R5.volume.fix(1333 * pyo.units.m**3)

The oxygen concentration in reactors 3-5 is injected through the CSTR injection reactors

In [10]:
# Injection rates to Reactions 3, 4 and 5
for j in m.fs.props_ASM1.component_list:
    if j != "S_O":
        # All components except S_O have no injection
        m.fs.R3.injection[:, :, j].fix(0)
        m.fs.R4.injection[:, :, j].fix(0)
        m.fs.R5.injection[:, :, j].fix(0)
# Then set injections rates for O2
m.fs.R3.outlet.conc_mass_comp[:, "S_O"].fix(1.72e-3)
m.fs.R4.outlet.conc_mass_comp[:, "S_O"].fix(2.43e-3)
m.fs.R5.outlet.conc_mass_comp[:, "S_O"].fix(4.49e-4)
# Oxygen concentration in reactors 3 and 4 is governed by mass transfer
m.fs.R3.KLa = 7.6
m.fs.R4.KLa = 5.7

Then we set up the separation, going through the splitters and the clarifier

In [11]:
# Set fraction of outflow from reactor 5 that goes to recycle
m.fs.SP5.split_fraction[:, "underflow"].fix(0.6)

# Clarifier
m.fs.CL1.split_fraction[0, "effluent", "H2O"].fix(0.48956)
m.fs.CL1.split_fraction[0, "effluent", "S_I"].fix(0.48956)
m.fs.CL1.split_fraction[0, "effluent", "S_S"].fix(0.48956)
m.fs.CL1.split_fraction[0, "effluent", "X_I"].fix(0.00187)
m.fs.CL1.split_fraction[0, "effluent", "X_S"].fix(0.00187)
m.fs.CL1.split_fraction[0, "effluent", "X_BH"].fix(0.00187)
m.fs.CL1.split_fraction[0, "effluent", "X_BA"].fix(0.00187)
m.fs.CL1.split_fraction[0, "effluent", "X_P"].fix(0.00187)
m.fs.CL1.split_fraction[0, "effluent", "S_O"].fix(0.48956)
m.fs.CL1.split_fraction[0, "effluent", "S_NO"].fix(0.48956)
m.fs.CL1.split_fraction[0, "effluent", "S_NH"].fix(0.48956)
m.fs.CL1.split_fraction[0, "effluent", "S_ND"].fix(0.48956)
m.fs.CL1.split_fraction[0, "effluent", "X_ND"].fix(0.00187)
m.fs.CL1.split_fraction[0, "effluent", "S_ALK"].fix(0.48956)

m.fs.CL1.surface_area.fix(1500 * pyo.units.m**2)

# Sludge purge separator
m.fs.SP6.split_fraction[:, "recycle"].fix(0.985)

[None]

The last thing required is the pressure for the recycle pump

In [12]:
# Outlet pressure from recycle pump
m.fs.P1.outlet.pressure.fix(101325)

### Step 2.2: Setup Anaerobic digester process

We will start by setting up the anaerobic digester process unit models and connectivity.

First, like above, we set up unit models. specifically the reactor. Which will have two different property blocks as it has two separate phases liquid and gas

In [13]:
m.fs.RADM = AD(
    liquid_property_package=m.fs.props_ADM1,
    vapor_property_package=m.fs.props_vap,
    reaction_package=m.fs.ADM1_rxn_props,
    has_heat_transfer=True,
    has_pressure_change=False,
)

In order to connect the ADM and ASM models translator blocks are required as they track different species

In [14]:
m.fs.asm_adm = Translator_ASM1_ADM1(
    inlet_property_package=m.fs.props_ASM1,
    outlet_property_package=m.fs.props_ADM1,
    reaction_package=m.fs.ADM1_rxn_props,
    has_phase_equilibrium=False,
    outlet_state_defined=True,
)

m.fs.adm_asm = Translator_ADM1_ASM1(
    inlet_property_package=m.fs.props_ADM1,
    outlet_property_package=m.fs.props_ASM1,
    reaction_package=m.fs.ADM1_rxn_props,
    has_phase_equilibrium=False,
    outlet_state_defined=True,
)

We the set up the separators and mixers 

In [15]:
m.fs.CL = Clarifier(
    property_package=m.fs.props_ASM1,
    outlet_list=["underflow", "effluent"],
    split_basis=SplittingType.componentFlow,
)

m.fs.TU = Thickener(property_package=m.fs.props_ASM1)
m.fs.DU = DewateringUnit(property_package=m.fs.props_ASM1)

m.fs.MX2 = Mixer(
    property_package=m.fs.props_ASM1, inlet_list=["feed_water1", "recycle1"], momentum_mixing_type=MomentumMixingType.equality
)
m.fs.MX3 = Mixer(
    property_package=m.fs.props_ASM1, inlet_list=["feed_water2", "recycle2"], momentum_mixing_type=MomentumMixingType.equality
)
m.fs.MX4 = Mixer(
    property_package=m.fs.props_ASM1, inlet_list=["thickener", "clarifier"], momentum_mixing_type=MomentumMixingType.equality
)

We add operating conditions to the unit models. We start with the primary clarifier

In [16]:
# Clarifier
m.fs.CL.split_fraction[0, "effluent", "H2O"].fix(0.993)
m.fs.CL.split_fraction[0, "effluent", "S_I"].fix(0.993)
m.fs.CL.split_fraction[0, "effluent", "S_S"].fix(0.993)
m.fs.CL.split_fraction[0, "effluent", "X_I"].fix(0.5192)
m.fs.CL.split_fraction[0, "effluent", "X_S"].fix(0.5192)
m.fs.CL.split_fraction[0, "effluent", "X_BH"].fix(0.5192)
m.fs.CL.split_fraction[0, "effluent", "X_BA"].fix(0.5192)
m.fs.CL.split_fraction[0, "effluent", "X_P"].fix(0.5192)
m.fs.CL.split_fraction[0, "effluent", "S_O"].fix(0.993)
m.fs.CL.split_fraction[0, "effluent", "S_NO"].fix(0.993)
m.fs.CL.split_fraction[0, "effluent", "S_NH"].fix(0.993)
m.fs.CL.split_fraction[0, "effluent", "S_ND"].fix(0.993)
m.fs.CL.split_fraction[0, "effluent", "X_ND"].fix(0.5192)
m.fs.CL.split_fraction[0, "effluent", "S_ALK"].fix(0.993)

We then set up the anaerobic digester operating conditions

In [17]:
# Anaerobic digester
m.fs.RADM.volume_liquid.fix(3400)
m.fs.RADM.volume_vapor.fix(300)
m.fs.RADM.liquid_outlet.temperature.fix(308.15)

Additionally, the dewatering unit includes an equation relating its hydraulic retention time to its volume and influent flowrate. We can choose to specify hydraulic retention time or the unit's volume to satisfy 0 degrees of freedom.

In [18]:
# Dewatering unit
m.fs.DU.hydraulic_retention_time.fix(1800 * pyo.units.s)
m.fs.DU.energy_electric_flow_vol_inlet[0] = 0.069 * pyo.units.kWh / pyo.units.m**3

Similarly, the thickener unit includes the same equation, as well as an equation relating the thickener's dimensions. Here, we fix hydraulic retention time and thickener diameter to satisfy 0 degrees of freedom.

In [19]:
# Thickener unit
m.fs.TU.hydraulic_retention_time.fix(86400 * pyo.units.s)
m.fs.TU.diameter.fix(10 * pyo.units.m)

We then again add arcs as streams linking the unit models

In [20]:
m.fs.stream2adm = Arc(
    source=m.fs.RADM.liquid_outlet, destination=m.fs.adm_asm.inlet
)
m.fs.stream6adm = Arc(source=m.fs.SP6.waste, destination=m.fs.TU.inlet)
m.fs.stream3adm = Arc(source=m.fs.TU.underflow, destination=m.fs.MX4.thickener)
m.fs.stream7adm = Arc(source=m.fs.TU.overflow, destination=m.fs.MX3.recycle2)
m.fs.stream9adm = Arc(source=m.fs.CL.underflow, destination=m.fs.MX4.clarifier)
m.fs.stream4adm = Arc(source=m.fs.adm_asm.outlet, destination=m.fs.DU.inlet)
m.fs.stream5adm = Arc(source=m.fs.DU.overflow, destination=m.fs.MX2.recycle1)
m.fs.stream01 = Arc(source=m.fs.FeedWater.outlet, destination=m.fs.MX2.feed_water1)
m.fs.stream02 = Arc(source=m.fs.MX2.outlet, destination=m.fs.MX3.feed_water2)
m.fs.stream03 = Arc(source=m.fs.MX3.outlet, destination=m.fs.CL.inlet)
m.fs.stream04 = Arc(source=m.fs.CL.effluent, destination=m.fs.MX1.feed_water)
m.fs.stream10adm = Arc(source=m.fs.MX4.outlet, destination=m.fs.asm_adm.inlet)
m.fs.stream1adm = Arc(source=m.fs.asm_adm.outlet, destination=m.fs.RADM.inlet)
pyo.TransformationFactory("network.expand_arcs").apply_to(m)

In [21]:
# Deactivate redundant constraints
m.mixers = (m.fs.MX1, m.fs.MX2, m.fs.MX3, m.fs.MX4, m.fs.MX6)
for mx in m.mixers:
    mx.pressure_equality_constraints[0.0, 2].deactivate()

# calculate and propagate scaling factors
iscale.calculate_scaling_factors(m.fs)

    

2024-12-05 15:29:15 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.FeedWater.properties[0.0].alkalinity
2024-12-05 15:29:15 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.MX1.feed_water_state[0.0].alkalinity
2024-12-05 15:29:15 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.MX1.recycle_state[0.0].alkalinity
2024-12-05 15:29:15 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.MX1.mixed_state[0.0].alkalinity
2024-12-05 15:29:15 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.R1.control_volume.properties_in[0.0].alkalinity
2024-12-05 15:29:15 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.R1.control_volume.properties_out[0.0].alkalinity
2024-12-05 15:29:15 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.R1.control_volume.volume
2024-12-05 15:29:15 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.R1.control_volume.rate_reaction_extent[0.0,R1]
2024-12-05 

# Step 3: Solve the square problem
## Step 3.1: Initialize the model

In [22]:
# Initialize flowsheet
# Apply sequential decomposition - 1 iteration should suffice
seq = SequentialDecomposition()
# seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Direct"
seq.options.iterLim = 1
seq.options.tear_set = [m.fs.stream2, m.fs.stream10adm]

G = seq.create_graph(m)
# Uncomment this code to see tear set and initialization order
order = seq.calculation_order(G)
print("Initialization Order")
for o in order:
    print(o[0].name)

# Initial guesses for flow into first reactor
tear_guesses1 = {
    "flow_vol": {0: 103531 / 24 / 3600},
    "conc_mass_comp": {
        (0, "S_I"): 0.028,
        (0, "S_S"): 0.012,
        (0, "X_I"): 1.532,
        (0, "X_S"): 0.069,
        (0, "X_BH"): 2.233,
        (0, "X_BA"): 0.167,
        (0, "X_P"): 0.964,
        (0, "S_O"): 0.0011,
        (0, "S_NO"): 0.0073,
        (0, "S_NH"): 0.0072,
        (0, "S_ND"): 0.0016,
        (0, "X_ND"): 0.0040,
    },
    "alkalinity": {0: 0.0052},
    "temperature": {0: 308.15},
    "pressure": {0: 101325},
}

tear_guesses2 = {
    "flow_vol": {0: 170 / 24 / 3600},
    "conc_mass_comp": {
        (0, "S_I"): 0.028,
        (0, "S_S"): 0.048,
        (0, "X_I"): 10.362,
        (0, "X_S"): 20.375,
        (0, "X_BH"): 10.210,
        (0, "X_BA"): 0.553,
        (0, "X_P"): 3.204,
        (0, "S_O"): 0.00025,
        (0, "S_NO"): 0.00169,
        (0, "S_NH"): 0.0289,
        (0, "S_ND"): 0.00468,
        (0, "X_ND"): 0.906,
    },
    "alkalinity": {0: 0.00715},
    "temperature": {0: 308.15},
    "pressure": {0: 101325},
}

# Pass the tear_guess to the SD tool
seq.set_guesses_for(m.fs.R1.inlet, tear_guesses1)
seq.set_guesses_for(m.fs.asm_adm.inlet, tear_guesses2)

Initialization Order
fs.R1
fs.R2
fs.R3
fs.DU
fs.MX2
fs.SP5
fs.CL1
fs.SP6
fs.MX6
fs.MX3
fs.CL
fs.MX1


We then run the initialization by creating a function to initialize each unit model and running it

In [23]:
def function(unit):
    unit.initialize(outlvl=idaeslog.INFO_HIGH)

seq.run(m, function)
# Deactivate redundant constraints that the initializer reactivated
m.mixers = (m.fs.MX1, m.fs.MX2, m.fs.MX3, m.fs.MX4, m.fs.MX6)
for mx in m.mixers:
    mx.pressure_equality_constraints[0.0, 2].deactivate()

2024-12-05 15:29:16 [INFO] idaes.init.fs.R1.control_volume.reactions: Initialization Complete.
2024-12-05 15:29:16 [INFO] idaes.init.fs.R1.control_volume.properties_out: State Released.
2024-12-05 15:29:16 [INFO] idaes.init.fs.R1.control_volume: Initialization Complete
2024-12-05 15:29:16 [INFO] idaes.init.fs.R1: Initialization Step 1 Complete.
2024-12-05 15:29:16 [INFO] idaes.init.fs.R1: Initialization Step 2 optimal - Optimal Solution Found.
2024-12-05 15:29:16 [INFO] idaes.init.fs.R1.control_volume.properties_in: State Released.
2024-12-05 15:29:16 [INFO] idaes.init.fs.R1: Initialization Complete: optimal - Optimal Solution Found
2024-12-05 15:29:16 [INFO] idaes.init.fs.FeedWater.properties: State Released.
2024-12-05 15:29:16 [INFO] idaes.init.fs.FeedWater.properties: Initialization Complete.
2024-12-05 15:29:16 [INFO] idaes.init.fs.FeedWater: Initialization Complete.
2024-12-05 15:29:16 [INFO] idaes.init.fs.asm_adm.properties_out: State Released.
2024-12-05 15:29:16 [INFO] idaes.i

## Step 3.2: Run solver
Solve the model by running the flowsheet using the ipopt solver.

In [24]:
solver = get_solver()
results = solver.solve(m, tee=True)

ipopt-watertap: ipopt with user variable scaling and IDAES jacobian constraint scaling
Ipopt 3.13.2: tol=1e-08
constr_viol_tol=1e-08
acceptable_constr_viol_tol=1e-08
bound_relax_factor=0.0
honor_original_bounds=no
nlp_scaling_method=user-scaling


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, 

We run an assertion to make sure the solver found the optimal solution

In [25]:
pyo.assert_optimal_termination(results)

## Step 3.3: report solution
we then report the treated water block

In [26]:
m.fs.Treated.report()


Unit : fs.Treated                                                          Time: 0.0
------------------------------------------------------------------------------------
    Stream Table
                                    Units             Inlet  
    Volumetric Flowrate        meter ** 3 / second    0.23889
    Molar Alkalinity             mole / meter ** 3     3.8096
    Mass Concentration S_I   kilogram / meter ** 3   0.061909
    Mass Concentration S_S   kilogram / meter ** 3 0.00087127
    Mass Concentration X_I   kilogram / meter ** 3  0.0054462
    Mass Concentration X_S   kilogram / meter ** 3 0.00020555
    Mass Concentration X_BH  kilogram / meter ** 3   0.010903
    Mass Concentration X_BA  kilogram / meter ** 3 0.00078876
    Mass Concentration X_P   kilogram / meter ** 3  0.0022565
    Mass Concentration S_O   kilogram / meter ** 3 0.00044900
    Mass Concentration S_NO  kilogram / meter ** 3   0.015456
    Mass Concentration S_NH  kilogram / meter ** 3 0.00091693
    Ma

# Part 2: Demonstrate optimization and sensitivity analysis over specific parameters of the flowsheet (pending)
The addition of unit capital costs and O&M costs is underway. Subsequently, an example of cost optimization of the steady-state BSM2 flowsheet will be presented.